In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # Add project root to Python path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from src.utils import load_train_data, load_test_data, resumetable, col_value_counts
train_df = load_train_data()
test_df = load_test_data()
submission = load

NameError: name 'load' is not defined

In [ ]:
all_data = pd.concat([train_df, test_df], ignore_index=True)
all_data = all_data.drop('target', axis=1)

In [ ]:
all_features = all_data.columns
all_features

In [ ]:
all_data

In [ ]:
all_data = all_data.drop(columns=['ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin', 'ps_ind_18_bin', 'ps_ind_14'])

In [ ]:
for feature in all_features:
    if ('calc' in feature):
        all_data = all_data.drop(columns=[feature])

In [ ]:
all_data.columns

In [ ]:
all_data_df = resumetable(all_data)

In [ ]:
missing_data = all_data_df[all_data_df['Missing'] > 0]
missing_data.sort_values(by='Missing', ascending=False)

In [ ]:
col_value_counts(all_data, 'ps_car_11')

In [ ]:
# 중앙값 대체 
continuois_features_to_impute = ['ps_reg_03', 'ps_car_14', 'ps_car_12']
for feature in continuois_features_to_impute:
    all_data[feature] = all_data[feature].replace(-1, np.nan)
    median_val = all_data[feature].median()
    all_data[feature] = all_data[feature].fillna(median_val)

In [ ]:
all_data_df = resumetable(all_data)
missing_data = all_data_df[all_data_df['Missing'] > 0]
missing_data.sort_values(by='Missing', ascending=False)

In [ ]:
all_data['ps_car_11'] = all_data['ps_car_11'].replace(-1, 3)

In [ ]:
all_data

In [ ]:
num_train_df = len(train_df)
X = all_data[:num_train_df]
X_test = all_data[num_train_df:]
y = train_df['target'].values

In [ ]:
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    L_mid = np.linspace(1 / n_samples, 1, n_samples)

    # 예측값에 대한 지니계수 
    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    G_pred = np.sum(L_mid - L_pred)

    # 예측이 완벽할 때 지니계수 
    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order)
    G_true = np.sum(L_mid - L_true)

    return G_pred / G_true

In [ ]:
# 기본적으로 lightgbm에서 내장 평가지표 지원하지만, 경진대회 요구 평가지표 코드
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True

In [ ]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=44)

In [ ]:
oof_val_preds = np.zeros(X.shape[0]) # OOF에서 훈련된 모델로 검증 데이터 타깃값을 예측한 확률 담을 배열
oof_test_preds = np.zeros(X_test.shape[0]) # OOF에서 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률 담을 배열
'''
	1.	전체 데이터 X.shape[0]은 100개
	2.	oof_val_preds = np.zeros(100) ← 먼저 100개의 빈 공간 마련
	3.	각 Fold 순회 시 마다:
    	• Fold 1: train 80개, val 20개
    	• 모델은 train으로 학습, val로 예측 → 예측 결과 20개 나옴
    	• 이 20개의 예측값을 원래 val_index 위치에 넣어줌
'''

In [ ]:
params = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'random_state': 44,
}
# params 딕셔너리에는 모델의 핵심 하이퍼 파라미터 사용
# 함수의 직접적인 인자로는 학습 실행과 관련된 제어요소 사용

import lightgbm as lgb

for idx, (train_idx, valid_idx) in enumerate(folds.split(X,y)):
    print(f'Fold {idx+1} / Fold {folds.n_splits}')
# split(X,y): folds 객체의 메서드 
# x와 y 데이터를 K개의 폴드로 분할 -> 실제 분할은 x, 데이터의 인덱스(위치 정보) 생성하여 반환
# enumerate(): 이터러블(iterable) 객체(folds.split(X,y)가 반환하는 이터레이터)를 순회하면서, 각 항목에 인덱스(순서 번호)를 부여
# for idx, (train_idx, valid_idx) in ...: (train_idx, valid_idx): folds.split(X,y)가 반환하는 튜플의 언패킹(unpacking)
# train_idx: 훈련 데이터 샘플들의 인덱스 배열, valid_idx: 검증 데이터 샘플들의 인덱스 배열
# idx: enumerate 함수가 부여하는 현재 반복의 인덱스 번호(0 ~ n_splits-1)
    
    # 훈련용/검증용 데이터 설정 
    X_train, y_train = X.iloc[train_idx], y[train_idx]
    X_valid, y_valid = X.iloc[valid_idx], y[valid_idx]

    # LightGBM 전용 데이터셋 설정 
    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)

    # LightGBM 모델 훈련
    lgb_model = lgb.train(
        params = params,
        train_set = dtrain,
        valid_sets = [dvalid],
        feval = gini,
        num_boost_round = 1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=100, verbose=False), 
            # early_stopping_rounds 대체
            lgb.log_evaluation(period=100) 
            # 학습 진행 상황 출력
            # Boosting Round(트리의 개수)가 100씩 증가하며 
            # 특정 번째 트리를 학습한 시점에서 검증 성능 출력
        ]
    )
    # 이 코드 마무리 되었을 때 fold 1에 대한 학습과 종료 

    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    # 이 코드 실행되었을 때 fold 1에 대한 예측치 모두 리스트에 채워짐
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'Fold {idx+1} gini score: {gini_score}\n')

    # 출력값 binary_logloss: params에 입력된 binary 인식하여 모델의 예측 확률과 실제 레이블의 일치도 측정, 낮을수록 정확


In [ ]:
print('OOF Data Gini:', eval_gini(y, oof_val_preds))

In [ ]:
submission = pd.DataFrame({'id': test_df['id'], 'target': oof_test_preds})
submission['target'] = oof_test_preds
submission.to_csv('submission.csv', index=False)